In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import fastf1
from fastf1 import plotting

In [ ]:
# Loading Raw Telemetry
leclerc_lap = laps.pick_driver("LEC").pick_fastest()
telemetry = leclerc_lap.get_telemetry()

tel = telemetry[['Time', 'Distance', 'Speed']].dropna().reset_index(drop=True)


In [ ]:
# Lap Time Reconstruction - Using distance increments and average velocity to reconstruct lap time, and comparing to official lap time
def reconstruct_lap_time(tel):
    tel = tel.copy()
    tel['Speed_ms'] = tel['Speed'] / 3.6
    tel['d_dist'] = tel['Distance'].diff()
    tel['dt'] = tel['d_dist'] / tel['Speed_ms']
    return tel['dt'].sum()

baseline_lap_time = reconstruct_lap_time(tel)
official_lap_time = leclerc_lap['LapTime'].total_seconds()

float(baseline_lap_time), float(official_lap_time), float(official_lap_time - baseline_lap_time)



In [ ]:
# Speed Perturbation Model
def apply_speed_perturbation(tel, start_dist, end_dist, speed_factor):
    tel_mod = tel.copy()
    mask = (tel_mod['Distance'] >= start_dist) & (tel_mod['Distance'] <= end_dist)
    tel_mod.loc[mask, 'Speed'] *= speed_factor
    return tel_mod


In [ ]:
# Sensitivty over Intervals
def segment_sensitivity_map(tel, segment_length=50, speed_factor=1.02):
    base_time = reconstruct_lap_time(tel)
    results = []

    max_dist = tel['Distance'].max()

    for start in np.arange(0, max_dist - segment_length, segment_length):
        end = start + segment_length
        tel_mod = apply_speed_perturbation(tel, start, end, speed_factor)
        new_time = reconstruct_lap_time(tel_mod)

        results.append({
            'segment_start': start,
            'segment_end': end,
            'delta_s': base_time - new_time
        })

    return pd.DataFrame(results)


In [ ]:
# Define what a Corner or Straight Is
def label_segments(tel, accel_threshold=2.5):
    tel_lab = tel.copy()
    tel_lab['Speed_ms'] = tel_lab['Speed'] / 3.6
    tel_lab['accel'] = tel_lab['Speed_ms'].diff()
    tel_lab['segment'] = np.where(
        tel_lab['accel'].abs() > accel_threshold,
        'corner',
        'straight'
    )
    return tel_lab


In [ ]:
# Labeling segments as Corners or Straights
def map_window_segments(sens_df, tel_labeled):
    labels = []

    for _, row in sens_df.iterrows():
        seg = tel_labeled[
            (tel_labeled['Distance'] >= row['segment_start']) &
            (tel_labeled['Distance'] <= row['segment_end'])
        ]['segment']

        labels.append(seg.mode().iloc[0] if not seg.empty else 'unknown')

    sens_df['segment'] = labels
    return sens_df


In [ ]:
# Final Sensitivty Analysis - Corners vs Straights
sens_df = segment_sensitivity_map(tel, segment_length=50, speed_factor=1.02)

telemetry_labeled = label_segments(tel)
sens_df = map_window_segments(sens_df, telemetry_labeled)

sens_df.head()


In [ ]:
# Lap Time Sensitivty - Time Gain over Intervals
plt.figure(figsize=(10,4))
plt.plot(sens_df['segment_start'], sens_df['delta_s'])
plt.xlabel("Distance (m)")
plt.ylabel("Lap Time Gain (s)")
plt.title("Lap-Time Sensitivity Map (+2% Speed, 50m Windows)")
plt.grid(True)
plt.show()


In [ ]:
# Lap Time Sensitivty Boxplot - Corner vs. Straight
corner_gains = sens_df[sens_df['segment'] == 'corner']['delta_s']
straight_gains = sens_df[sens_df['segment'] == 'straight']['delta_s']

plt.figure(figsize=(8,4))
plt.boxplot([corner_gains, straight_gains], labels=['Corners', 'Straights'])
plt.ylabel("Lap Time Gain (s)")
plt.title("Corner vs Straight Speed Sensitivity")
plt.grid(True, axis='y')
plt.show()


In [ ]:
# Asymmetry Test - +5% vs. -5%
def asymmetry_test(tel, start, end, factor):
    tel = tel.copy()
    tel['Speed_ms'] = tel['Speed'] / 3.6
    tel['d_dist'] = tel['Distance'].diff()
    tel['dt'] = tel['d_dist'] / tel['Speed_ms']
    
    base_time = tel['dt'].sum()

    # Speed up
    tel_up = tel.copy()
    mask = (tel_up['Distance'] >= start) & (tel_up['Distance'] <= end)
    tel_up.loc[mask, 'Speed_ms'] *= factor
    tel_up['dt'] = tel_up['d_dist'] / tel_up['Speed_ms']
    gain = base_time - tel_up['dt'].sum()

    # Slow down
    tel_down = tel.copy()
    tel_down.loc[mask, 'Speed_ms'] /= factor
    tel_down['dt'] = tel_down['d_dist'] / tel_down['Speed_ms']
    loss = tel_down['dt'].sum() - base_time

    return gain, loss

gain, loss = asymmetry_test(tel, 1000, 1050, 1.05)
print(f"Gain: {gain:.4f}s, Loss: {loss:.4f}s")

plt.bar(['Gain (+5%)', 'Loss (-5%)'], [gain, loss])
plt.ylabel('Lap Time Change (s)')
plt.title('Speed Asymmetry Test')
plt.show()

In [ ]:
# Global Comparison Plot - Lap-Time Sensitivity vs Speed
plt.figure(figsize=(10,5))

# Sensitivity plot (top layer)
plt.plot(sens_df['segment_start'], sens_df['delta_s'],
         color='red', label='Lap-Time Sensitivity (+2%)', linewidth=2)

# Speed profile (bottom layer, scaled for visibility)
# We'll scale speed to match the plot visually
speed_scaled = (tel['Speed'] / 3.6) * (sens_df['delta_s'].max() / (tel['Speed'].max() / 3.6))
plt.plot(tel['Distance'], speed_scaled, color='blue', label='Speed (scaled)', alpha=0.7)

plt.xlabel("Distance (m)")
plt.ylabel("Lap-Time Gain / Scaled Speed")
plt.title("Global Lap-Time Sensitivity vs Speed Profile")
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# Export and Freeze Results
sens_df.to_csv("sensitivity_map.csv", index=False)
telemetry_labeled.to_csv("telemetry_labeled.csv", index=False)
